$H0:$ 기사를 수치화 하면 가계부실선행지수와 유의한 회귀분석할 수 있다  
&nbsp;&nbsp; $y = f(x)$  
&nbsp;&nbsp; $x$ : 월별기사 doc2vec  
&nbsp;&nbsp; $y$ : 가계부실선행지수 1표준편차 초과여부

In [1]:
from konlpy.tag import Okt
okt = Okt()

In [2]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [3]:
from navsch import *

naver_search : 기사검색 후 결과를 data frame으로 산출  
naver_search_d : 기간별 기사검색 후 결과를 data frame으로 산출

#### 1.기사를 검색한다  
&nbsp;&nbsp;- 2002년 1월~2019년 3월 월별

In [44]:
b_rng = pd.date_range('2002-01-01','2019-04-01',freq='MS').strftime('%Y.%m.%d').tolist()[:-1]
e_rng = pd.date_range('2002-01-01','2019-04-01',freq='M').strftime('%Y.%m.%d').tolist()

In [51]:
%%time
dfs = []
for beg, end in zip(b_rng,e_rng):
    s = time.time()
    dfs.append(naver_search_d('가계부채',beg,end,pages=1))
    e = time.time() - s
    print('{}~{} done, {} elapsed'.format(beg,end,time.strftime('%H:%M:%S',time.gmtime(e))))
newsdf = pd.concat(dfs,axis=0)    
# Wall time: 1h 12min 22s

가계부채 2002 search done, 00:00:18 elapsed
2002.01.01~2002.01.31 done, 00:00:18 elapsed
가계부채 2002 search done, 00:00:23 elapsed
2002.02.01~2002.02.28 done, 00:00:23 elapsed
가계부채 2002 search done, 00:00:18 elapsed
2002.03.01~2002.03.31 done, 00:00:18 elapsed
가계부채 2002 search done, 00:00:18 elapsed
2002.04.01~2002.04.30 done, 00:00:18 elapsed
가계부채 2002 search done, 00:00:19 elapsed
2002.05.01~2002.05.31 done, 00:00:19 elapsed
가계부채 2002 search done, 00:00:20 elapsed
2002.06.01~2002.06.30 done, 00:00:20 elapsed
가계부채 2002 search done, 00:00:20 elapsed
2002.07.01~2002.07.31 done, 00:00:20 elapsed
가계부채 2002 search done, 00:00:19 elapsed
2002.08.01~2002.08.31 done, 00:00:19 elapsed
가계부채 2002 search done, 00:00:20 elapsed
2002.09.01~2002.09.30 done, 00:00:20 elapsed
가계부채 2002 search done, 00:00:22 elapsed
2002.10.01~2002.10.31 done, 00:00:22 elapsed
가계부채 2002 search done, 00:00:21 elapsed
2002.11.01~2002.11.30 done, 00:00:21 elapsed
가계부채 2002 search done, 00:00:20 elapsed
2002.12.01~2002.12.31 don

In [52]:
newsdf.to_pickle('data/newsdf_03_19.pkl')

In [2]:
import gc

In [3]:
gc.collect()

0

In [4]:
newsdf = pd.read_pickle('data/newsdf_03_19.pkl')

In [6]:
newsdf.shape

(2019, 5)

#### 2.기사를 월별로 합친다

In [6]:
newsdf.head(3)

,ym,qry,date,title,text
0,200201,가계부채,None,"""가계부채 과도한 증가는 `금융불안' 초래""<금융연>",`신용카드 부실이 도화선'(서울=연합뉴스) 한승호기자 = 가계부채의 과도한 증가는 ...
1,200201,가계부채,None,"금융연구원, 31일 가계부채 세미나 개최",머니투데이\n\n[머니투데이] 한국금융연구원(KIF)은 오는 31일 오후 2시 은행...
2,200201,가계부채,None,선진국 가계.기업 모두 빚더미,"= '절약국가 일본' 옛말 가계 부채 세계최고 =\n\n\n\n미국은 물론 영국, ..."


In [7]:
title = newsdf.groupby(['ym','qry'])['title'].apply(lambda x: x.sum())

In [8]:
text = newsdf.groupby(['ym','qry'])['text'].apply(lambda x: x.sum())

In [15]:
ndf = pd.concat([title,text],axis=1).reset_index()

In [16]:
ndf.head()

,ym,qry,title,text
0,200201,가계부채,"""가계부채 과도한 증가는 `금융불안' 초래""<금융연>금융연구원, 31일 가계부채 세...",`신용카드 부실이 도화선'(서울=연합뉴스) 한승호기자 = 가계부채의 과도한 증가는 ...
1,200202,가계부채,"가계부채 335조 ‘사상 최고’…4년만에 배 가까이 증가""가계부채 우려는 과장......",지난해 말 현재 가계부채가 330조원을 넘어 사상 최고를 기록했다.이에따라 가구당 ...
2,200203,가계부채,"가계부채증가 우려할 만한 수준인가?가계부채 위험수위윤 재경차관 ""가계부채 증가속도 ...","1. 송인호 기자, 최근 가계부채증가 속도에 대한 우려의 목소리가 높습니다...외국..."
3,200204,가계부채,"가계부채 GDP의 64% 달해…KDI""과다한 가계부채 주의 필요""<전경련 원로.자문...",가계부문 금융부채가 국내총생산(GDP)에서 차지하는 비중이 지난해 64.8%를 기록...
4,200205,가계부채,"산업은행, 올 경제성장률 6.2%로 상향 전망""한국 월드컵 선전하면 증시 상승""-F...","미경제 둔화, 환율하락 등 경제불안요인 상존(서울=연합뉴스) 한승호기자 = 산업은행..."


In [17]:
ndf.to_pickle('data/ndf.pkl')

In [18]:
del newsdf

#### 3.월별 기사를 tokenize 한다

In [4]:
ndf = pd.read_pickle('data/ndf.pkl')

In [5]:
ndf.head(3)

,ym,qry,title,text
0,200201,가계부채,"""가계부채 과도한 증가는 `금융불안' 초래""<금융연>금융연구원, 31일 가계부채 세...",`신용카드 부실이 도화선'(서울=연합뉴스) 한승호기자 = 가계부채의 과도한 증가는 ...
1,200202,가계부채,"가계부채 335조 ‘사상 최고’…4년만에 배 가까이 증가""가계부채 우려는 과장......",지난해 말 현재 가계부채가 330조원을 넘어 사상 최고를 기록했다.이에따라 가구당 ...
2,200203,가계부채,"가계부채증가 우려할 만한 수준인가?가계부채 위험수위윤 재경차관 ""가계부채 증가속도 ...","1. 송인호 기자, 최근 가계부채증가 속도에 대한 우려의 목소리가 높습니다...외국..."


tok_p: Adjective, Adverb, Noun 인것만 꺼낸다

In [62]:
pos_list = ['Adjective','Adverb','Noun']
tok_p = lambda doc: ['/'.join(t) for t in okt.pos(doc) if t[1] in pos_list] # okt.pos(doc): (사스, Noun), 이거 사스/Noun

In [77]:
s = time.time()
ndf['tok_tit'] = ndf.title.apply(tok_p)
e = time.time()-s
print('title tokenize complete, {} elapsed'.format(time.strftime('%H:%M:%S',time.gmtime(e))))
ndf['tok_txt'] = ndf.text.apply(tok_p)
e = time.time()-s
print('text tokenize complete, {} elapsed'.format(time.strftime('%H:%M:%S',time.gmtime(e))))

title tokenize complete, 00:00:05 elapsed
text tokenize complete, 00:06:09 elapsed


In [78]:
ndf.to_pickle('data/ndf_tok.pkl')

In [8]:
ndf.head()

,ym,qry,title,text,tok_tit,tok_txt
0,200201,가계부채,"""가계부채 과도한 증가는 `금융불안' 초래""<금융연>금융연구원, 31일 가계부채 세...",`신용카드 부실이 도화선'(서울=연합뉴스) 한승호기자 = 가계부채의 과도한 증가는 ...,"[""/Punctuation, 가계부채/Noun, 과도/Noun, 한/Josa, 증가...","[`/Punctuation, 신용카드/Noun, 부실/Noun, 이/Josa, 도화..."
1,200202,가계부채,"가계부채 335조 ‘사상 최고’…4년만에 배 가까이 증가""가계부채 우려는 과장......",지난해 말 현재 가계부채가 330조원을 넘어 사상 최고를 기록했다.이에따라 가구당 ...,"[가계부채/Noun, 335조/Number, ‘/Foreign, 사상/Noun, 최...","[지난해/Noun, 말/Noun, 현재/Noun, 가계부채/Noun, 가/Josa,..."
2,200203,가계부채,"가계부채증가 우려할 만한 수준인가?가계부채 위험수위윤 재경차관 ""가계부채 증가속도 ...","1. 송인호 기자, 최근 가계부채증가 속도에 대한 우려의 목소리가 높습니다...외국...","[가계부채/Noun, 증가/Noun, 우려/Noun, 할/Verb, 만/Noun, ...","[1/Number, ./Punctuation, 송인/Noun, 호/Noun, 기자/..."
3,200204,가계부채,"가계부채 GDP의 64% 달해…KDI""과다한 가계부채 주의 필요""<전경련 원로.자문...",가계부문 금융부채가 국내총생산(GDP)에서 차지하는 비중이 지난해 64.8%를 기록...,"[가계부채/Noun, GDP/Alpha, 의/Noun, 64%/Number, 달해/...","[가계/Noun, 부문/Noun, 금융/Noun, 부채/Noun, 가/Josa, 국..."
4,200205,가계부채,"산업은행, 올 경제성장률 6.2%로 상향 전망""한국 월드컵 선전하면 증시 상승""-F...","미경제 둔화, 환율하락 등 경제불안요인 상존(서울=연합뉴스) 한승호기자 = 산업은행...","[산업/Noun, 은행/Noun, ,/Punctuation, 올/Verb, 경제성장...","[미경/Noun, 제/Noun, 둔화/Noun, ,/Punctuation, 환율/N..."


In [12]:
tt = ndf.tok_tit.sum()

In [39]:
m = []
p = []
for t in tt:
    m.append(t.split('/')[0])
    p.append(t.split('/')[1])

In [40]:
mp = pd.DataFrame({'pos':p,'morph':m})

In [41]:
mp.head()

,pos,morph
0,Punctuation,""""
1,Noun,가계부채
2,Noun,과도
3,Josa,한
4,Noun,증가


In [31]:
pos = set([t.split('/')[1] for t in tt])
pos = list(pos)

In [33]:
del pos[1]

In [35]:
del pos[4]

In [36]:
pos

['VerbPrefix',
 'Verb',
 'Josa',
 'Number',
 'URL',
 'Adverb',
 'Punctuation',
 'Modifier',
 'Adjective',
 'Conjunction',
 'Exclamation',
 'Eomi',
 'Determiner',
 'Foreign',
 'Suffix',
 'Noun',
 'Alpha']

In [48]:
mp.columns

Index(['pos', 'morph'], dtype='object')

In [61]:
1!=1

False

In [60]:
mp.groupby('pos').apply(lambda x: x.sample(1))
# Adjective, Adverb, Noun 형용사,부사,명사만 가지고 토크나이즈 한다 ==> 위쪽 코드 중 tok_p 수정 

,,pos,morph
pos,,,
,9082,,
4분,4402,4분,2
Adjective,18872,Adjective,성한
Adverb,6830,Adverb,함께
Alpha,427,Alpha,FT
Conjunction,16478,Conjunction,그리고
Determiner,13942,Determiner,한
Eomi,6604,Eomi,는다
Exclamation,5294,Exclamation,어휴


#### 4.월별 기사를 doc2vec 한다

In [79]:
ndf = pd.read_pickle('data/ndf_tok.pkl')

doc2vec용으로 가공(TaggedDocument)  
&nbsp;TaggedDocument에서 tags는 words그룹의 식별자 역할을 한다  
&nbsp;$example:$ TaggedDocuments(words=['had','bankruptcy','year'],tags=['Credit Report'])

In [ ]:
tok_p = lambda doc: ['/'.join(t) for t in okt.pos(doc) if t[1] in pos_list]

In [81]:
ndf.head()

,ym,qry,title,text,tok_tit,tok_txt
0,200201,가계부채,"""가계부채 과도한 증가는 `금융불안' 초래""<금융연>금융연구원, 31일 가계부채 세...",`신용카드 부실이 도화선'(서울=연합뉴스) 한승호기자 = 가계부채의 과도한 증가는 ...,"[가계부채/Noun, 과도/Noun, 증가/Noun, 금융/Noun, 불안/Noun...","[신용카드/Noun, 부실/Noun, 도화선/Noun, 서울/Noun, 연합뉴스/N..."
1,200202,가계부채,"가계부채 335조 ‘사상 최고’…4년만에 배 가까이 증가""가계부채 우려는 과장......",지난해 말 현재 가계부채가 330조원을 넘어 사상 최고를 기록했다.이에따라 가구당 ...,"[가계부채/Noun, 사상/Noun, 최고/Noun, 배/Noun, 가까이/Noun...","[지난해/Noun, 말/Noun, 현재/Noun, 가계부채/Noun, 사상/Noun..."
2,200203,가계부채,"가계부채증가 우려할 만한 수준인가?가계부채 위험수위윤 재경차관 ""가계부채 증가속도 ...","1. 송인호 기자, 최근 가계부채증가 속도에 대한 우려의 목소리가 높습니다...외국...","[가계부채/Noun, 증가/Noun, 우려/Noun, 만/Noun, 수준/Noun,...","[송인/Noun, 호/Noun, 기자/Noun, 최근/Noun, 가계부채/Noun,..."
3,200204,가계부채,"가계부채 GDP의 64% 달해…KDI""과다한 가계부채 주의 필요""<전경련 원로.자문...",가계부문 금융부채가 국내총생산(GDP)에서 차지하는 비중이 지난해 64.8%를 기록...,"[가계부채/Noun, 의/Noun, 과/Noun, 다한/Noun, 가계부채/Noun...","[가계/Noun, 부문/Noun, 금융/Noun, 부채/Noun, 국내총생산/Nou..."
4,200205,가계부채,"산업은행, 올 경제성장률 6.2%로 상향 전망""한국 월드컵 선전하면 증시 상승""-F...","미경제 둔화, 환율하락 등 경제불안요인 상존(서울=연합뉴스) 한승호기자 = 산업은행...","[산업/Noun, 은행/Noun, 경제성장률/Noun, 로/Noun, 상향/Noun...","[미경/Noun, 제/Noun, 둔화/Noun, 환율/Noun, 하락/Noun, 등..."


In [82]:
tagdoc = lambda txt, ym: [TaggedDocument(words=txt,tags=ym)]

In [92]:
ndf['tag_tit'] = ndf.apply(lambda x: tagdoc(x.tok_tit,x.ym),axis=1)

In [93]:
ndf['tag_txt'] = ndf.apply(lambda x: tagdoc(x.tok_txt,x.ym),axis=1)

In [94]:
ndf.tag_tit[0]

[TaggedDocument(words=['가계부채/Noun', '과도/Noun', '증가/Noun', '금융/Noun', '불안/Noun', '초래/Noun', '금융/Noun', '연/Noun', '금융/Noun', '연구원/Noun', '가계부채/Noun', '세미나/Noun', '개최/Noun', '선진국/Noun', '가계/Noun', '기업/Noun', '모두/Noun', '빚/Noun', '더미/Noun', '금감원/Noun', '은행/Noun', '가계/Noun', '여신/Noun', '급증/Noun', '오늘/Noun', '기사/Noun', '차/Noun', '메모/Noun', '목/Noun', '개인/Noun', '대출/Noun', '폭증/Noun', '대비/Noun', '금융/Noun', '세미나/Noun', '한경/Noun', '밀레니엄/Noun', '포럼/Noun', '차/Noun', '회의/Noun', '토론/Noun', '지상/Noun', '중계/Noun', '올해/Noun', '경제정책/Noun', '방향/Noun', '신용/Noun', '불량/Noun', '위험수위/Noun', '금융/Noun', '세미나/Noun', '인물/Noun', '동정/Noun', '홍일/Noun', '흥사단/Noun', '통일/Noun', '포럼/Noun', '회장/Noun', '등/Noun', '동정/Noun', '고/Noun', '건/Noun', '서울시장/Noun'], tags='200201')]

doc2vec에 token load

$word2vec$: CBOW ==> $doc2vec$: **DM(Distributed Memory)** dm=1  
$word2vec$: Skip-Gram ==> $doc2vec$: **DBOW(Distributed Bag of Words)** dm=0

#### 4.1 토큰 벡터화 (word embedding)

In [97]:
import multiprocessing
cores = multiprocessing.cpu_count()
cores

4

In [101]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

In [102]:
# doc2vec 인스턴스 생성(dm=0; dbow ~ skip-gram이랑 비슷한거)
model_dbow = Doc2Vec(dm=0, 
                     vector_size=30, 
                     negative=5, 
                     hs=0, 
                     min_count=2, 
                     sample = 0, 
                     workers=cores)

In [103]:
model_dbow.build_vocab([x for x in tqdm(ndf.tag_tit[0])])

100%|████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


In [109]:
from sklearn import utils

In [110]:
%%time
for epoch in range(10):
    model_dbow.train(utils.shuffle([x for x in tqdm(ndf.tag_tit[0])]),
                    total_examples=len(ndf.tag_tit[0]),
                    epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


Wall time: 91 ms


In [113]:
ndf.tag_tit[0]

[TaggedDocument(words=['가계부채/Noun', '과도/Noun', '증가/Noun', '금융/Noun', '불안/Noun', '초래/Noun', '금융/Noun', '연/Noun', '금융/Noun', '연구원/Noun', '가계부채/Noun', '세미나/Noun', '개최/Noun', '선진국/Noun', '가계/Noun', '기업/Noun', '모두/Noun', '빚/Noun', '더미/Noun', '금감원/Noun', '은행/Noun', '가계/Noun', '여신/Noun', '급증/Noun', '오늘/Noun', '기사/Noun', '차/Noun', '메모/Noun', '목/Noun', '개인/Noun', '대출/Noun', '폭증/Noun', '대비/Noun', '금융/Noun', '세미나/Noun', '한경/Noun', '밀레니엄/Noun', '포럼/Noun', '차/Noun', '회의/Noun', '토론/Noun', '지상/Noun', '중계/Noun', '올해/Noun', '경제정책/Noun', '방향/Noun', '신용/Noun', '불량/Noun', '위험수위/Noun', '금융/Noun', '세미나/Noun', '인물/Noun', '동정/Noun', '홍일/Noun', '흥사단/Noun', '통일/Noun', '포럼/Noun', '회장/Noun', '등/Noun', '동정/Noun', '고/Noun', '건/Noun', '서울시장/Noun'], tags='200201')]

In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressorsdef vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [ ]:
dvec3 = list(map(lambda x: model3.infer_vector(x), df.token_review_n))
cname = ['dv'+str(i) for i in np.arange(1,51)]
dvec3 = pd.DataFrame(dvec3,columns = cname)

In [117]:
ndf.tag_tit[0]

[TaggedDocument(words=['가계부채/Noun', '과도/Noun', '증가/Noun', '금융/Noun', '불안/Noun', '초래/Noun', '금융/Noun', '연/Noun', '금융/Noun', '연구원/Noun', '가계부채/Noun', '세미나/Noun', '개최/Noun', '선진국/Noun', '가계/Noun', '기업/Noun', '모두/Noun', '빚/Noun', '더미/Noun', '금감원/Noun', '은행/Noun', '가계/Noun', '여신/Noun', '급증/Noun', '오늘/Noun', '기사/Noun', '차/Noun', '메모/Noun', '목/Noun', '개인/Noun', '대출/Noun', '폭증/Noun', '대비/Noun', '금융/Noun', '세미나/Noun', '한경/Noun', '밀레니엄/Noun', '포럼/Noun', '차/Noun', '회의/Noun', '토론/Noun', '지상/Noun', '중계/Noun', '올해/Noun', '경제정책/Noun', '방향/Noun', '신용/Noun', '불량/Noun', '위험수위/Noun', '금융/Noun', '세미나/Noun', '인물/Noun', '동정/Noun', '홍일/Noun', '흥사단/Noun', '통일/Noun', '포럼/Noun', '회장/Noun', '등/Noun', '동정/Noun', '고/Noun', '건/Noun', '서울시장/Noun'], tags='200201')]

In [119]:
a = [model_dbow.infer_vector(doc.words) for doc in ndf.tag_tit[0]]

In [126]:
ndf.tag_txt[0][0].words[:5] # 이걸 가지고 벡터라이즈 한다

['신용카드/Noun', '부실/Noun', '도화선/Noun', '서울/Noun', '연합뉴스/Noun']

#### 가계부실선행지수

In [127]:
busil_idx = pd.read_clipboard(sep='\t')

In [130]:
busil_idx.head()

,ym,cycle,tcycle,가계소득지표,가계소비지표,차입수준지표
0,201207,104.4,103.2,0.109,0.448,0.321
1,201208,104.4,103.2,0.121,0.387,0.367
2,201209,104.4,103.2,0.137,0.327,0.407
3,201210,104.3,103.1,0.157,0.262,0.441
4,201211,104.2,103.0,0.175,0.200,0.467


In [131]:
busil_idx.to_pickle('data/busil_idx.pkl')

#### 5.월별기사와 가계부실선행지수를 합친다